<a target="_blank" href="https://colab.research.google.com/github/bettercodepaul/data2day_2023_polars/blob/main/data2day_2023_Polars_Teil_3.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Polars: The Turbo Boost for Dataframes - Part 3

Important links as a reminder:

- Homepage of Polars: https://www.pola.rs/
- User Guide: https://pola-rs.github.io/polars/user-guide/
- API Reference: https://pola-rs.github.io/polars/py-polars/html/reference/

## Installation + Set-Up

In [ ]:
import urllib.request
import os.path

In [ ]:
REQUIREMENTS_URL = "https://github.com/bettercodepaul/data2day_2023_polars/raw/main/requirements.txt"
urllib.request.urlretrieve(REQUIREMENTS_URL, os.path.basename(REQUIREMENTS_URL))

In [ ]:
# don't forget that you might need to restart the kernel
!pip install -qr requirements.txt

In [ ]:
import polars as pl

In [ ]:
# output up to 60 characters per column and do not abbreviate floating point numbers
pl.Config(fmt_str_lengths=60, fmt_float="full")

In [ ]:
# download CSV data
CSV_DATA_URL = "https://github.com/bettercodepaul/data2day_2023_polars/raw/main/spotify-charts-2017-2021-global-top200.csv.gz"
LOCAL_CSV_DATA_FILE_NAME = os.path.basename(CSV_DATA_URL)
urllib.request.urlretrieve(CSV_DATA_URL, LOCAL_CSV_DATA_FILE_NAME)
REGION_DATA_URL = "https://github.com/bettercodepaul/data2day_2023_polars/raw/main/region-info.csv"
LOCAL_REGION_DATA_FILE_NAME = os.path.basename(REGION_DATA_URL)
urllib.request.urlretrieve(REGION_DATA_URL, LOCAL_REGION_DATA_FILE_NAME)

In [ ]:
# download Parquet data
BIG_DATA_URL = "https://github.com/bettercodepaul/data2day_2023_polars/releases/download/data-parquet/spotify-charts-2017-2021.parquet"
LOCAL_BIG_DATA_FILE_NAME = os.path.basename(BIG_DATA_URL)
urllib.request.urlretrieve(BIG_DATA_URL, LOCAL_BIG_DATA_FILE_NAME)

In [ ]:
# download excercises and utility functions
EXERCISES_URL = "https://github.com/bettercodepaul/data2day_2023_polars/raw/main/data2day_exercises_en.py"
urllib.request.urlretrieve(EXERCISES_URL, os.path.basename(EXERCISES_URL))

In [ ]:
from data2day_exercises_en import *

## When.Then.Otherwise

Sometimes you want to calculate an expression in certain cases like this and in other cases like that.

For this there are the methods `when.then.otherwise`, which correspond to an `if.then.else`.

In [ ]:
df = pl.read_csv("spotify-charts-2017-2021-global-top200.csv.gz", try_parse_dates=True)

It then works like this:

In [ ]:
(df
    .with_columns(
        pl.when(pl.col("date").dt.day().eq(14) & pl.col("date").dt.month().eq(2))
        .then(pl.col("streams"))
        .otherwise(pl.lit(0))
        .alias("valentinesStreams")
    )
    .filter(pl.col("title").eq("Starboy") & pl.col("date").dt.week().eq(7))
    .select("date", "streams", "valentinesStreams")
    .head(5)
)

We could also recreate the trend column ourselves:

In [ ]:
(df
    .join(
        # Determine rank from previous day
        df.select("url", pl.col("date").dt.offset_by("1d"), pl.col("rank").alias("previous_rank")),
        how="left",
        on=["url", "date"]
    )
    .with_columns(
        pl.when(pl.col("rank").lt(pl.col("previous_rank")))
        .then(pl.lit("MOVE_UP"))
        .otherwise(
            pl.when(pl.col("rank").gt(pl.col("previous_rank")))
            .then(pl.lit("MOVE_DOWN"))
            .otherwise(
                pl.when(pl.col("rank").eq(pl.col("previous_rank")))
                .then(pl.lit("SAME_POSITION"))
                .otherwise(pl.lit("NEW_ENTRY"))
            )
        ).alias("myTrend")
    )
    .select("title", "artist", "date", "trend", "myTrend")
    .sample(10)
)

If it is only about replacing certain, single values with others, `map_dict` can also be very handy.

In [ ]:
mapping = {
    "SAME_POSITION": "➡️",
    "NEW_ENTRY": "🆕",
    "MOVE_UP": "⬆️",
    "MOVE_DOWN": "⬇️"
}
(df
  .with_columns(pl.col("trend")
  .map_dict(mapping).alias("trendSymbol"))
  .group_by("trendSymbol")
  .count()
)

## Custom Expressions

### Please do not: `map_*`

The worst possible way to smuggle your own functions into a Polars query are the various map methods `map_rows`, `map_batches`, `map_elements` and `map_groups`, which execute a UDF (User Defined Function).

This should be avoided because performance suffers greatly.

### Expression Factories




A more elegant way to get modular code is to use custom methods that create new expressions and can be called e.g. with the `pipe` method.

Here is an example for the method `sum` for which there is no `min_count` parameter in Polars (but there is in Pandas). The `min_count` parameter determines how many values must be present at least for the sum to be formed.

In [ ]:
def sum(expr: pl.Expr, min_count=0) -> pl.Expr:
    if min_count > 0:
        return pl.when(
            expr.is_not_null().sum().ge(pl.lit(min_count))
        ).then(expr.sum())
    else:
        return expr.sum()

In [ ]:
# direct call to the method
pl.DataFrame({
    "value": [42, 43, None], 
}).select(
    sum(pl.col("value"), min_count=2).alias("min_count=2"),
    sum(pl.col("value"), min_count=3).alias("min_count=3")
)

In [ ]:
# call via pipe
pl.DataFrame({
    "value": [42, 43, None], 
}).select(
    pl.col("value").pipe(sum, min_count=2).alias("min_count=2"),
    pl.col("value").pipe(sum, min_count=3).alias("min_count=3")
)

We can also register such methods in a separate namespace.

In [ ]:
@pl.api.register_expr_namespace("special")
class Special:
    def __init__(self, expr: pl.Expr):
        self._expr = expr

    def sum(self, min_count=0) -> pl.Expr:
        if min_count > 0:
            return pl.when(
                self._expr.is_not_null().sum().ge(pl.lit(min_count))
            ).then(self._expr.sum())
        else:
            return self._expr.sum()

Now we can call the method within its own namespace "special".

In [ ]:
pl.DataFrame({
    "value": [42, 43, None], 
}).select(
    pl.col("value").special.sum(min_count=2).alias("min_count=2"),
    pl.col("value").special.sum(min_count=3).alias("min_count=3")
)

Further information is available here:

- [Expr.pipe](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.pipe.html)
- [DataFrame.pipe](https://pola-rs.github.io/polars/py-polars/html/reference/dataframe/api/polars.DataFrame.pipe.html)
- [Extending the API](https://pola-rs.github.io/polars/py-polars/html/reference/api.html)

## Lazy vs. Eager

Until now we have always used Polars in "eager mode". Each function call directly resulted in an operation on the data.

This has advantages when debugging queries, but prevents many optimizations that Polars can only use in "lazy mode".

There are two options for the "lazy mode".

### Eager Load + Lazy Query

If a data set is not too large, we can load it completely into memory, as we already know.

In [ ]:
df = pl.read_csv("spotify-charts-2017-2021-global-top200.csv.gz")
type(df)

By calling the `lazy` method we then switch to the "lazy mode". The execution of the query is now stopped and with each further call the query is only "formulated".

In [ ]:
lazy_df = df.lazy()
type(lazy_df)

In [ ]:
# for a lazy dataframe the unoptimized query tree is printed
lazy_df.select("artist", "title").filter(pl.col("artist").eq("Dua Lipa"))

The plan is read from bottom to top. The Greek letters are from relational algebra. The letter π stands for the operation projection (`select`), σ for the operation selection (`filter`).

- Table π */9; σ -; means that all nine columns are read and no selection is made.
- π 2/9 means that two out of nine columns are projected 
- FILTER BY is the filter from our query

In [ ]:
# with the method show_graph() we can output the optimized query
lazy_df.select("artist", "title").filter(pl.col("artist").eq("Dua Lipa")).show_graph()

Both projection and selection happen earlier in the optimized query plan.

The query is finally executed when we call the `collect` method. The result is then a normal dataframe again.

In [ ]:
result = lazy_df.select("artist", "title").filter(pl.col("artist").eq("Dua Lipa")).collect()
result.sample(2)

In [ ]:
type(result)

By doing this, Polars can perform optimizations before executing the query.

A selection of optimizations can be found here: https://pola-rs.github.io/polars/user-guide/lazy/optimizations/

### Lazy Load + Query

If it is not worth loading a data set completely into memory, we can also delay loading the data by using the IO methods named `scan_*` instead of `write_*`.

This works e.g. for files in CSV (`scan_csv`) and Parquet (`scan_parquet`) formats, but not for compressed CSVs.

So far we have always worked with a small dataset containing only the global top-200 charts (362k lines, 64 MB).

We can now switch to the real dataset that contains the Top-200 and Viral-50 charts for 70 different regions (26m rows, 4 GB).

In [ ]:
df = pl.scan_parquet("spotify-charts-2017-2021.parquet")

Due to the optimized queries, only the data that is really needed is loaded from the file.

In [ ]:
(df
    .select("artist", "title")
    .filter(pl.col("artist").eq("Dua Lipa"))
).show_graph()

Depending on the query, certain optimizations cannot be performed because they would change the result...

In [ ]:
(df
    .head(2)
    .select("artist", "title")
    .filter(pl.col("artist").eq("Dua Lipa"))
).show_graph()

Sometimes the query optimizer also misses potential optimizations...

In [ ]:
naive_query = (df
    .group_by("artist")
    .agg(pl.col("title").n_unique())
    .filter(pl.col("artist").eq("Dua Lipa"))
)
naive_query.show_graph()

In [ ]:
%%timeit
naive_query.collect()

We manually optimize that filtering should be done first, which significantly speeds up the query.

In [ ]:
optimized_query = (df
    .filter(pl.col("artist").eq("Dua Lipa"))
    .group_by("artist")
    .agg(pl.col("title").n_unique())
)
optimized_query.show_graph()

In [ ]:
%%timeit
optimized_query.collect()

However, the optimizer is also under constant development. See for this specific case e.g. https://github.com/pola-rs/polars/issues/11678

### Streaming

If the final result or even intermediate results of a query do not fit into RAM anymore, Polars has a "streaming mode" which can significantly reduce the required RAM.

If only the intermediate results are the problem, the streaming mode can be activated with `collect(streaming=True)`. The final result must then fit into the RAM.

To write also a final result that does not fit into RAM anymore to disk, the methods `sink_parquet`, `sink_csv` and `sink_ipc` can be used.

In [ ]:
# If the Jupyter kernel crashed, restart and run this line
import polars as pl
df = pl.scan_parquet("spotify-charts-2017-2021.parquet")

In [ ]:
# fraction is the fraction of rows and influences the memory requirement
# 0.003 ~ 4 GB (should run with 8 GB RAM)
# 0.005 ~ 10 GB (should run with 16 GB RAM)
# 0.008 ~ 26 GB (should run with 32 GB RAM)
# 0.010 ~ 41 GB (should run with 64 GB RAM)
# 0.015 ~ 92 GB (should run with 128 GB RAM)
fraction = 0.008
row_count = round(26173514*fraction)
high_mem_query = (
    df.head(row_count).join(df.head(row_count), on="artist")
    .filter(
        pl.col("url").ne(pl.col("url_right")) &
        pl.col("date").gt(pl.col("date_right")) &
        pl.col("trend").eq("NEW_ENTRY") &
        pl.col("trend_right").eq("NEW_ENTRY")
    )
    .group_by("url").agg((pl.col("date") - pl.col("date_right")).min().alias("durationBetweenNewEntries"))
    .select(pl.col("durationBetweenNewEntries").mean())
)
print(f"Cross-product of {row_count:_} rows would contain {row_count**2:_} rows.")
print(f"Estimated size for the intermediate join result is {6e-10*row_count**2:.2f} GB.")

In [ ]:
# try different fractions with streaming=False and streaming=True
high_mem_query.collect(streaming=False)

In [ ]:
When a query can be executed in streaming mode it will be inside a "pipeline" node. If some nodes can not be streamed you will see them outside of the "pipeline" node.

In [ ]:
high_mem_query.show_graph(streaming=True)

## Exercises

In [ ]:
df = (
    pl.scan_parquet("spotify-charts-2017-2021.parquet")
    .with_columns(pl.col("streams").cast(pl.Int64))
)

### Question 21

In [ ]:
q21.question()

In [ ]:
q21_df = ...

In [ ]:
q21.check(q21_df)

### Question 22

In [ ]:
q22.question()

In [ ]:
q22_df = ...

In [ ]:
q22.check(q22_df)

### Question 23

In [ ]:
q23.question()

In [ ]:
q23_df = ...

In [ ]:
q23.check(q23_df)

### Question 24

In [ ]:
q24.question()

In [ ]:
q24_df = ...

In [ ]:
q24.check(q24_df)

### Question 25

In [ ]:
q25.question()

In [ ]:
region_df = pl.scan_csv("region-info.csv")
xmasYears_per_continent = ...


In [ ]:
q25_df = ...

In [ ]:
q25.check(q25_df)